In [2]:
pip install shap

  Using cached shap-0.45.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (24 kB)
  Using cached slicer-0.0.7-py3-none-any.whl.metadata (3.7 kB)
Using cached shap-0.45.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (538 kB)
Using cached slicer-0.0.7-py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow==2.12

  Using cached tensorflow-2.12.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.4.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached jax-0.4.26-py3-none-any.whl.metadata (23 kB)
  Using cached keras-2.12.0-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached tensorboard-2.12.3-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached

In [4]:
pip install sagemaker-experiments

  Using cached sagemaker_experiments-0.1.45-py3-none-any.whl.metadata (10 kB)
Using cached sagemaker_experiments-0.1.45-py3-none-any.whl (42 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
import os
from datetime import datetime  # Make sure this import is included
import boto3
from sagemaker.session import Session
import pandas as pd
import numpy as np
import tensorflow as tf
from sagemaker.experiments.run import Run,load_run
from sagemaker.session import Session
import argparse
import os
from datetime import datetime
import boto3
from io import StringIO
import pandas as pd
import botocore
import matplotlib.pyplot as plt

import seaborn as sns

bucket = 'multiclass-balanced-two-million'
file_key = 'merged-multiclass-balanced-two-million.csv'

s3_input_path = f's3://{bucket}/{file_key}'

s3_input = sagemaker.inputs.TrainingInput(s3_data=s3_input_path, content_type='csv')
s3_output_path = f's3://{bucket}/lstm_training_output_confusion/' 
    


# Use the same experiment name as defined in the training script
experiment_name = "Experiments-LSTM-comparison-SHAP"
role = sagemaker.get_execution_role()
region = sagemaker.Session().boto_region_name
aws_region = os.environ.get('AWS_REGION', 'us-east-2') 
boto_session= boto3.setup_default_session(region_name=aws_region)
sagemaker_session = Session(boto_session)
s3 = boto3.client("s3")


# hyperparameter_ranges = {
#     'learning_rate': ContinuousParameter(0.0001, 0.01),
#     'dropout': ContinuousParameter(0.0, 0.5),
#     'optimizer': CategoricalParameter(['RMSprop', 'Adam', 'Adagrad', 'Adadelta', 'Adamax', 'Nadam']),
#     'activation': CategoricalParameter(['tanh', 'relu', 'sigmoid']),
#     'epochs':IntegerParameter(5, 50),
#     'units': IntegerParameter(50, 200),
#     'batch_size': IntegerParameter(64, 512)
# }



with Run(
    experiment_name=experiment_name ,
    sagemaker_session=sagemaker_session ,
) as run:

        estimator = TensorFlow(
            # metric_definitions=metric_definitions,
            script_mode=True,
            entry_point="load_run_visualization.py",
            role=role,
            hyperparameters={
                              'learning_rate': 0.0001,
                               'dropout': 0.3,
                               'optimizer': 'Adagrad',
                               'activation':'sigmoid',
                               'epochs':22,
                               'units':139,
                               'batch_size': 281,
                           },
            framework_version="2.12",
            py_version='py310',
            instance_type='ml.g4dn.16xlarge',
            instance_count=1,
            keep_alive_period_in_seconds=10,
            environment={"REGION": region},

            output_path=s3_output_path
        )



        # estimator.fit()
        estimator.fit({'train': s3_input})
